# BM25(Best Matching 25)
tf-idfの進化系(tf-idfと違い、文章の長さと単語の重要度を柔軟に重みづけ可能)
$$ score(q,d) = \sum_i{idf(q_i)}\times\frac{(k_1+1)f(q_i,d)}{f(q_i,d)+k_1(1-b+b\frac{\#d}{\textrm{avgdl}})} $$
$k_1,b$はBM25を制御するためのパラメータ<br>
$\#d$は文章d内の単語数、$\textrm{avgdl}$はaverage of document lengthで平均のドキュメントの長さを表す。<br>
（つまり$\frac{\#d}{\textrm{avgdl}}$は平均の文章に対する相対的な長さを表している。）<br>
$f(q_i,d)$は文書$d$の中の単語$q_i$の量であり、tfや$q_iのカウント$、$\sqrt{q_iのカウント}$が用いられる。

## BM25が今もなぜ使われているか
- シンプルで実装が簡単
- 計算効率が良い
- 多くの計算エンジン(ElasticSearch, Lucene)に使用されている
- 単純に精度が高い

## BM25の欠点
- 特定のキーワードが入っていないと検索されない（逆に固有名詞にはとても強い）
- クエリの文脈は考慮されない

## BM25の解読をする
簡単のため、$$ score(q,d) = \sum_i{idf(q_i)}\times\phi $$ とし、
$$ \phi = \frac{(k_1+1)f}{f+k_1(1-b+b\frac{\#d}{\textrm{avgdl}})} $$
を考える。
1. #d=avgdlのときを考える。
この場合、$$ \phi = \frac{(k_1+1)f}{f+k_1} $$ となる。<br>
f=0のとき、$\phi$=0<br>
f=1のとき、$\phi$=1<br>
f=2のとき、$\phi$=2-$\frac{2}{2+k_1}< 2$<br>
fは$\phi$に対して単調増加<br>
fがどれだけ大きくなっても$\phi$の上限は$k_1+1$(#d=avgdlではないときも)

![BM25](img/BM25.avif "サンプル")

2. #d $ \neq $ avgdlのとき
分母の調整項の部分を考える。
$$ 1-b+b\frac{\#d}{\textrm{avgdl}} = (1-b) \times 1 + b \times \frac{\#d}{\textrm{avgdl}}  $$
つまり1と$ \frac{\#d}{\textrm{avgdl}} $を(1-b):bの割合で足している（凸関数みたいな重み付き平均）。<br>
これが分母に来ているので、
- 相対的な長さが大きいほど、分母は大きくなり$\phi$は小さくなる。<br>
- 相対的な長さが小さいほど、分母は小さくなり$\phi$は大きくなる。<br>
相対的な長さによる影響はbで調整が可能。

3. 要約
- $b$:相対的な長さの影響を決める。
- $k_1$:$\phi$の最大値を$k_1+1$に決める。
- また、$b=0.75$ ほど、$ k_1 = 1.2$くらいが良く用いられる。

## 参考url
- https://www.youtube.com/watch?v=_HSOX0oh2ns (おすすめ)
- https://qiita.com/KokiSakano/items/2a0f4c45caaa09cf1ab9

In [25]:
# from rank_bm25 import BM25Okapi
import numpy as np
import math

# コーパスとクエリの定義
corpus = [
    "hello world hello",
    "hello good morning",
    "hello world",
    "python BM25 implementation"
]
query = "hello world"

# テキストをトークン化
def tokenize(text):
    return text.split()

tokenized_corpus = [tokenize(doc) for doc in corpus]
tokenized_query = tokenize(query)

# BM25の実装
# def compute_bm25(corpus, query, k1, b):
#     bm25 = BM25Okapi(corpus, k1=k1, b=b)
#     scores = bm25.get_scores(query)
#     return scores

def compute_idf(corpus):
    N = len(corpus)
    idf = {}
    df = {}
    for doc in corpus:
        unique_term = set(doc)
        for term in unique_term:
            df[term] = df.get(term, 0) + 1
    for term, freq in df.items():
        idf[term] = math.log((N - freq + 0.5)/ (freq + 0.5) + 1)
    return idf

def compute_bm25(corpus, query, k1=1.2, b=0.75):
    idf = compute_idf(tokenized_corpus)
    avgdl = sum(len(doc) for doc in corpus) / len(corpus)
    scores = []

    for doc in corpus:
        score = 0.0
        doc_len = len(doc)
        term_freqs = {}
        for term in doc:
            term_freqs[term] = term_freqs.get(term, 0) + 1

        for q in query:
            if q in term_freqs:
                f = term_freqs[q]
                numerator = f * (k1 + 1)
                denominator = f + k1 * (1 - b + b * doc_len / avgdl)
                score += idf.get(q, 0) * (numerator / denominator)
        scores.append(score)

    return np.array(scores)

# 初期パラメータ
k1 = 1.2
b = 0.75

# BM25スコアの計算
bm25_scores = compute_bm25(tokenized_corpus, tokenized_query, k1, b)

print("BM25 Scores with k1={} and b={}:".format(k1, b))
print(bm25_scores)

BM25 Scores with k1=1.2 and b=0.75:
[1.14649461 0.3438858  1.18166025 0.        ]
